## OSMOSIS DEX DATA

Data source: https://api-osmosis.imperator.co/swagger/#/

We need: 
- 6-month average daily volume
- 6-month average TVL
- Largest pool concentration
- Liquidity pools with > 1mn TVL
- 6 month average FDV:TVL
- Monthly active users
- Average trade execution time
- Implied Slippage
- MEV

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime as dt 
from dateutil.relativedelta import relativedelta

import warnings
warnings.filterwarnings(action="ignore")

### 6-month average daily volume

In [2]:
def get_6month_average_volume():
    url = 'https://api-osmosis.imperator.co/volume/v2/historical/chart'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()
        df = pd.DataFrame(data)
        df['time'] = pd.to_datetime(df['time'])
        return df['value'].loc[df['time'] >= (dt.datetime.today() - relativedelta(months = 6))].sum()/len(pd.date_range(start = (dt.datetime.today().replace(hour=0,minute=0,second=0,microsecond=0) - relativedelta(months = 6)), end = (dt.datetime.today().replace(hour=0,minute=0,second=0,microsecond=0)), freq= 'D'))
    else:
        raise KeyError('Endpoint could be fucked!')

In [3]:
get_6month_average_volume()

79269653.93604946

### 6-month average TVL

In [4]:
def get_6month_average_TVL():
    url = 'https://api-osmosis.imperator.co/liquidity/v2/historical/chart'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()
        df = pd.DataFrame(data)
        df['time'] = pd.to_datetime(df['time'])
        return df['value'].loc[df['time'] >= (dt.datetime.today() - relativedelta(months = 6))].sum()/len(pd.date_range(start = (dt.datetime.today().replace(hour=0,minute=0,second=0,microsecond=0) - relativedelta(months = 6)), end = (dt.datetime.today().replace(hour=0,minute=0,second=0,microsecond=0)), freq= 'D'))
    else:
        raise KeyError('Endpoint could be fucked!')

In [236]:
avgtvl = get_6month_average_TVL()
avgtvl

1063402584.373773

### Largest pool concentration 

In [6]:
def largest_pool_concentration_TVL():
    url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=true'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()

        df = pd.DataFrame()
        for key in data.keys():
            tdf = pd.DataFrame(data[key])
            tdf['pool_id'] = key
            df = df.append(tdf)
        
        ddf = df.drop_duplicates(subset='pool_id')
        ddf['proportionTVL'] = ddf['liquidity'] / ddf['liquidity'].sum()
        
        largest_prop = ddf['proportionTVL'].max()
        top10_prop = ddf['proportionTVL'].nlargest(10).sum()

        return [largest_prop, top10_prop]
    else:
        raise KeyError('Endpoint could be fucked!')
    

In [7]:
largest_pool_concentration_TVL()

[0.43124148628714704, 0.8271723792313572]

In [8]:
def largest_pool_concentration_7d_volume():
    url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=true'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()

        df = pd.DataFrame()
        for key in data.keys():
            tdf = pd.DataFrame(data[key])
            tdf['pool_id'] = key
            df = df.append(tdf)
        
        ddf = df.drop_duplicates(subset='pool_id')
        ddf['proportionVol'] = ddf['volume_7d'] / ddf['volume_7d'].sum()
        
        largest_prop = ddf['proportionVol'].max()
        top10_prop = ddf['proportionVol'].nlargest(10).sum()

        return [largest_prop, top10_prop]
    else:
        raise KeyError('Endpoint could be fucked!')

In [9]:
largest_pool_concentration_7d_volume()

[0.33596960571310786, 0.8595311130473487]

### Liquidity pools with > 1mil USD  

In [10]:
def no_pools_gte_1mUSD():
    url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=true'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()

        df = pd.DataFrame()
        for key in data.keys():
            tdf = pd.DataFrame(data[key])
            tdf['pool_id'] = key
            df = df.append(tdf)
        
        ddf = df.drop_duplicates(subset='pool_id')

        return len(ddf.loc[ddf['liquidity'] >= 1000000])
    else:
        raise KeyError('Endpoint could be fucked!')

In [11]:
no_pools_gte_1mUSD()

16

### Fee Spread

In [30]:
def liquid_pair_fee_spread():
    url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=false'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()

        df = pd.DataFrame()
        for key in data.keys():
            tdf = pd.DataFrame(data[key])
            tdf['pool_id'] = key
            df = df.append(tdf)
    
        mini = df['fees'].apply(lambda x: float(x.replace('%', ''))).min()
        maxi = df['fees'].apply(lambda x: float(x.replace('%', ''))).max()

        return [mini, maxi]
    else:
        raise KeyError('Endpoint could be fucked!')

In [31]:
liquid_pair_fee_spread()

[0.04, 0.534]

### 6 month average FDV/TVL 

In [277]:
#FDV is the blocker

In [283]:
url = 'https://api.cosmostation.io/v1/status'
response = requests.get(url)

In [284]:
response.ok

True

In [285]:
data = response.json()

In [286]:
data['total_supply_tokens']

{'supply': [{'denom': 'ibc/0030B0AA8A1B9028703B1D17B965FBCBE0136571B854410BFEC3475B1DC765A1',
   'amount': '43210'},
  {'denom': 'ibc/0276EC3A17E2BADE81821D73FE20CB6D71487F151FEE36E159962B8721942F9D',
   'amount': '1000000'},
  {'denom': 'ibc/03FDDDFA51B58B17520629DD2082A9D8B2D72FA06BD060E660006ACF9D9AF58C',
   'amount': '100000000000000000'},
  {'denom': 'ibc/04F26F89E89475EBEDA75428CD312CF2DF569A3ED0C999CAC1CCA151FF30694A',
   'amount': '5000000000'},
  {'denom': 'ibc/07912C24004932CD561B1751562B22EA787F31F9821568B88F55A8F51D326722',
   'amount': '5000'},
  {'denom': 'ibc/080D7604D9C1960A3562E1AE4E8EA8F30B7813810AE30660D41DD6770B1C5929',
   'amount': '2000000000'},
  {'denom': 'ibc/08822F65D0CE3FD27B35B9E99046CBAB72D731C7991BBC8586E8916F78721FEF',
   'amount': '100000'},
  {'denom': 'ibc/08834A76F4E5AED08690916F61EA12AA71CFD636BBA328062027DF9FA620B7E3',
   'amount': '1'},
  {'denom': 'ibc/0C754B423F325B3C8DEBD837CE7C4CF0D4B0E60BEB710E8C8C2E231769DC357C',
   'amount': '103000'},
  {'d

In [259]:
total_supply = pd.DataFrame(data['total_supply_tokens']['supply'])

In [260]:
url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=false'
response = requests.get(url, headers= {'accept': 'application/json'})
if response.ok:
    data = response.json()

    df = pd.DataFrame()
    for key in data.keys():
        tdf = pd.DataFrame(data[key])
        tdf['pool_id'] = key
        df = df.append(tdf)

    ddf = df.drop_duplicates(subset='pool_id')
    ids = list(ddf['pool_id'].loc[ddf['liquidity'].isin(ddf['liquidity'].nlargest(10))])

In [278]:
tdf = df.loc[df['pool_id'] == ids[1]]

In [279]:
tdf

,symbol,amount,denom,coingecko_id,liquidity,liquidity_24h_change,volume_24h,volume_24h_change,volume_7d,price,fees,main,pool_id
0,JUNO,1.305051e+06,ibc/46B44899322F3CD854D2D46DEEF881958467CDD4B3...,juno-network,6.388559e+06,-3.877527,205954.926023,-51.765194,1.667714e+06,2.447628,0.3%,True,497
1,OSMO,4.124375e+06,uosmo,osmosis,6.388559e+06,-3.877527,205954.926023,-51.765194,1.667714e+06,0.774488,0.3%,True,497


In [281]:
total_supply.loc[total_supply['denom'] == tdf['denom'].iloc[0]]

,denom,amount


### Montly active users

### Average Trade Exection Time

Block time: https://www.mintscan.io/cosmos/validators

6.86s 

### Implied Slippage 

In [225]:
def balancer_slippage(tdf, dollar_value):
    ep = tdf['amount'].iloc[-1]/tdf['amount'].iloc[0]
    token0amountin = dollar_value/(tdf['price'].iloc[0])
    token1out = token0amountin*(1-(tdf['amount'].iloc[0]/(tdf['amount'].iloc[0]+tdf['amount'].iloc[-1])))
    nep = (tdf['amount'].iloc[-1] - token1out)/(tdf['amount'].iloc[0]+ token0amountin)
    slipage = abs(ep/nep -1)
    return slipage

In [226]:
def get_slippage():
    url = 'https://api-osmosis.imperator.co/pools/v2/all?low_liquidity=false'
    response = requests.get(url, headers= {'accept': 'application/json'})
    if response.ok:
        data = response.json()

        df = pd.DataFrame()
        for key in data.keys():
            tdf = pd.DataFrame(data[key])
            tdf['pool_id'] = key
            df = df.append(tdf)

        ddf = df.drop_duplicates(subset='pool_id')
        ids = list(ddf['pool_id'].loc[ddf['liquidity'].isin(ddf['liquidity'].nlargest(10))])

        slip1000 = []
        slip10000 = []
        slip100000 = []
        for i in range(len(ids)):
            tdf = df.loc[df['pool_id'] == ids[i]]
            slip1000.append(balancer_slippage(tdf, 1000))
            slip10000.append(balancer_slippage(tdf, 10000))
            slip100000.append(balancer_slippage(tdf, 100000))
            
        return  slip1000, slip10000, slip100000

    else:
        raise KeyError('Endpoint could be fucked!')


In [228]:
s1, s2, s3 = get_slippage()

In [230]:
np.mean(s1)

0.0006580321466434169

In [233]:
np.mean(s2)

0.006603324677011501

In [234]:
np.mean(s3)

0.06846529943753894

### MEV